In [1]:
import os
%pwd

'c:\\Users\\Hp\\Desktop\\Wine_Quality_End_To_End\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\Hp\\Desktop\\Wine_Quality_End_To_End'

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [34]:
from WineQuality.constant import *
from WineQuality.utils.common import read_yaml, create_directories

In [39]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH
                ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) ->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMNS
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )
        return model_trainer_config
        

In [40]:
import pandas as pd
import os
from WineQuality import logger
from sklearn.linear_model import ElasticNet
import joblib

In [41]:
class ModelTraner:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config
    
    def training(self):
        train_df = pd.read_csv(self.config.train_data_path)
        test_df = pd.read_csv(self.config.test_data_path)
        
        x_train = train_df.drop([self.config.target_column], axis=1)
        x_test = test_df.drop([self.config.target_column], axis=1)
        
        y_train = train_df[[self.config.target_column]]
        y_tets = test_df[[self.config.target_column]]
        
        lr = ElasticNet(alpha = self.config.alpha,
                        l1_ratio=self.config.l1_ratio,
                        random_state=42)
        lr.fit(x_train, y_train)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [42]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTraner(config=model_trainer_config)
    model_trainer.training()
    
except Exception as e:
    raise e

[2023-11-23 00:35:43,277:  INFO:  common : yaml file : config\config.yaml loaded successfully]
[2023-11-23 00:35:43,285:  INFO:  common : yaml file : params.yaml loaded successfully]
[2023-11-23 00:35:43,293:  INFO:  common : yaml file : schema.yaml loaded successfully]
[2023-11-23 00:35:43,293:  INFO:  common : Created directories at: artifacts]
[2023-11-23 00:35:43,301:  INFO:  common : Created directories at: artifacts/model_trainer]
